In [78]:
# Dependencies
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import pymongo
from selenium import webdriver

In [79]:
#import driver for chrome
driver = webdriver.Chrome()

In [80]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables to reference later.
driver = webdriver.Chrome()
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&blank_scope=Latest'
driver.get(url)
soup = bs4(driver.page_source, 'html.parser')


In [81]:
#use beautifulsoup to scrape the news title
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&blank_scope=Latest'
driver.get(url)
soup = bs4(driver.page_source, 'html.parser')
try:
    news_title = soup.find_all(class_="content_title")[1].text
except:
    news_title = []
try:
    news_desc = soup.find_all(class_="article_teaser_body")[0].text
except:
    news_desc = []

In [82]:
# Visit the url for JPL Featured Space Image here.
url2 ='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
driver.get(url2)
soup2 = bs4(driver.page_source, 'html.parser')  

# pull the featured image and clean
featured_image_url = soup2.find('article', class_="carousel_item")['style']
featured_image_url = featured_image_url.replace("background-image: url('","").replace("');","")

image_url = f'https://www.jpl.nasa.gov{featured_image_url}'
print(image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18886-1920x1200.jpg


In [83]:
# Find the featured image source and use
featured_image_url = soup2.find('a',class_="button fancybox")['data-fancybox-href']
image_url = print(f"{url2}{featured_image_url}")
print(image_url)

https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/mediumsize/PIA18886_ip.jpg
None


In [84]:
# Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.
url3 ='https://twitter.com/marswxreport?lang=en'
driver.get(url3)
soup3 = bs4(driver.page_source, 'html.parser')


In [85]:
#pull the latest tweet with beautifulsoup
mars_weather = soup3.findAll(class_="tweet-text")[0].text
print(mars_weather)

None


In [86]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
url4 ='https://space-facts.com/mars/'
data = pd.read_html(url4)
df= data[0]
# Use Pandas to convert the data to a HTML table string.
html = df.to_html()
print(html)


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorded

In [87]:
# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
url5 ='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response5 = requests.get(url5)
soup5 = bs4(response5.text, 'html.parser')


In [88]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [89]:
# Define database and collection
db = client.nhl_db
collection = db.mars

In [90]:
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

url5 ='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response5 = requests.get(url5)
soup5 = bs4(response5.text, 'html.parser')

hemispheres = soup5.find_all(class_="item")
titles = []
urls = []
for hemi in hemispheres:
    name = hemi.find('h3').text
    src = hemi.find('img')['src']    
    src= f"https://astrogeology.usgs.gov{src}"
    titles.append(name)
    urls.append(src)

        
mars_data = {"news_title": news_title
    , "news_desc": news_desc
    , "image_url": image_url
    , "mars_weather": mars_weather
    , "html":html
    , "titles": titles
    , "urls": urls}

print(urls)
print(mars_data)

['https://astrogeology.usgs.gov/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png', 'https://astrogeology.usgs.gov/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png', 'https://astrogeology.usgs.gov/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png', 'https://astrogeology.usgs.gov/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png']
{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin ", 'news_desc': "Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.", 'image_url': None, 'mars_weather': None, 'html': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Dia

In [91]:
#create a displat for the hemisphere data collected

hemispheres = soup5.find_all(class_="item")
for hemi in hemispheres:
    title = hemi.find('h3').text
    src = hemi.find('a')
    src= f"{url5}{src['href']}"
    hemi_dict= {'title': title, 'URL': src}
    print(title)
    print(src)
    print('--------')
    post = {'title': title, 'URL': src}
    collection.insert_one(post)

Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/cerberus_enhanced
--------
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/schiaparelli_enhanced
--------
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/syrtis_major_enhanced
--------
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/valles_marineris_enhanced
--------


In [92]:
# Display the MongoDB records created above
mars = db.mars.find()
for m in mars:
    print(m)

{'_id': ObjectId('5ed1569ac1c74b969690fd3c'), 'title': 'Cerberus Hemisphere Enhanced', 'URL': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/cerberus_enhanced'}
{'_id': ObjectId('5ed1569ac1c74b969690fd3d'), 'title': 'Schiaparelli Hemisphere Enhanced', 'URL': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/schiaparelli_enhanced'}
{'_id': ObjectId('5ed1569ac1c74b969690fd3e'), 'title': 'Syrtis Major Hemisphere Enhanced', 'URL': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/syrtis_major_enhanced'}
{'_id': ObjectId('5ed1569ac1c74b969690fd3f'), 'title': 'Valles Marineris Hemisphere Enhanced', 'URL': 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars/search/map/Mars/Viking/valles_marineris_enhanced'}
{'_id': ObjectId('5ed26781c33acfba84076671'), 'title': 'Cerberus Hemisphere 